In [12]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
import os
openai_api_key = os.getenv('OPENAI_API_KEY')
openai.api_key =openai_api_key

In [14]:
# CSV 파일 불러오기 (첫 번째 행 제외)
file_path='../관광지-하은님1700.csv'

df_ex = pd.read_csv(file_path)

In [15]:
df_ex.head()

,KO,ENG,JPN
0,부용공원,Buyong Park,芙蓉公園
1,북새말공원,Buksaemal Park,北セマル公園
2,비둘기근린공원,Pigeon Neighborhood Park,ハト近隣公園
3,비룡성당,Biryong Cathedral,飛龍聖堂
4,비발디공원,Vivaldi Park,ビバルディ公園


In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
    
def generate_sentences_from_csv(df, openai_api_key):
    
    llm = ChatOpenAI(model_name="gpt-4o-mini", openai_api_key=openai_api_key)
    
    results = []
    
    # prompts
    korean_prompt = PromptTemplate.from_template(
        """
        You are a professional Korean language educator.
        Create 1 natural Korean sentence that include the word '{word}' without numbering them.
        Provide only the sentence without the original sentence, additional examples, symbols, or punctuation explanations.
        """
        )
    english_prompt = PromptTemplate.from_template(
        """
        You are a professional English translator.
        Translate the following Korean sentence into English using the word '{english_hint}':  \n{korean_sentences}\n.
        Provide only the translated text without the original sentence, additional examples, symbols, or punctuation explanations.
        Keep the same order, each on a new line.        
        """)
    japanese_prompt = PromptTemplate.from_template(
        """
        You are a professional Japanese translator.
        Translate the following Korean sentence into Japanese using the word '{japanese_hint}':  \n{korean_sentences}\n.
        Provide only the translated text without the original sentence, additional examples, symbols, or punctuation explanations.
        Keep the same order, each on a new line.
        """)
    
    # chains
    korean_chain = LLMChain(llm=llm, prompt=korean_prompt, output_key="korean_sentences")
    english_chain = LLMChain(llm=llm, prompt=english_prompt, output_key="english_translations")
    japanese_chain = LLMChain(llm=llm, prompt=japanese_prompt, output_key="japanese_translations")
    
    
    # Sequential chain(chain묶음)
    translation_chain = SequentialChain(
        chains=[korean_chain, english_chain, japanese_chain],
        input_variables=["word", "english_hint", "japanese_hint"],
        output_variables=["korean_sentences", "english_translations", "japanese_translations"]
    )
    
    for index, row in df.iterrows():
        korean_word = row.iloc[0]  # 한국어
        english_hint = row.iloc[1]  # 영어
        japanese_hint = row.iloc[2]  # 일본어
    
        response = translation_chain(
            {"word": korean_word, "english_hint": english_hint, "japanese_hint": japanese_hint}
        )         
        
        # # 5 result split
        # korean_sentences = response["korean_sentences"].strip().split("\n")
        # english_translations = response["english_translations"].strip().split("\n")
        # japanese_translations = response["japanese_translations"].strip().split("\n")

        # if len(korean_sentences) == len(english_translations) == len(japanese_translations) == 1:
        #     for i in range(1):
        #         results.append([korean_sentences[i], english_translations[i], japanese_translations[i]])
        # else:
        #     print(f"Warning: Mismatch in sentence count for word '{korean_word}'")
    
    
        korean_sentence = response["korean_sentences"]
        english_translation = response["english_translations"]
        japanese_translation = response["japanese_translations"]

        results.append([korean_sentence, english_translation, japanese_translation])
    
    
    result_df = pd.DataFrame(results, columns=[ "Korean Sentence", "English Translation", "Japanese Translation"])
    result_df.to_csv("travel_3.csv", index=False, encoding="utf-8-sig")
    
    print("저장됨v")
    

generate_sentences_from_csv(df_ex, openai_api_key)

저장됨v
